In [67]:
#import required packages

import streamlit as st 
import plotly.express as px
import pandas as pd
import numpy as np
from bokeh.plotting import figure, show, output_file
from bokeh.models import ColumnDataSource
from bokeh.palettes import Viridis3
from bokeh.models import HoverTool

In [2]:
#define function to plot oas by ward

def plot_wards(df, column='', string='', agg='',title=''):
    df=df[df.MEASURES_NAME== 'Percent']
    df=df[df[column].str.contains(string)==True]
    fig = px.choropleth(df.dissolve(by='ward_name', aggfunc ={'OBS_VALUE':agg}),
                   geojson=df.dissolve(by='ward_name', aggfunc ={'OBS_VALUE':agg}).geometry,
                   locations=df.dissolve(by='ward_name',aggfunc ={'OBS_VALUE':agg}).index,
                   color="OBS_VALUE",
                   color_continuous_scale = 'viridis_r',
                   projection="mercator",
                   hover_name=df.dissolve(by='ward_name',aggfunc ={'OBS_VALUE':agg}).index )
    fig.update_geos(fitbounds="locations", visible=False)
    fig.update_layout(coloraxis_colorbar=dict(title=title))
    fig.update_traces(hovertemplate='Ward name: <b>%{location}</b> <br>Percentage: <b>%{z:.2f}%</b>')
    st.plotly_chart(fig,use_container_width = True)

In [3]:
#define function to merge spatial data with variable data

def merge_spatial_data(gdf, df, left_on="", right_on=""):
    gdf=gdf.merge(df, left_on=left_on, right_on=right_on)
    return gdf

In [4]:
#read in aggregated oa by ward dataset from main python file 

#merged_wd_oa = st.session_state['merged_wd_oa']

In [5]:
#read in deprivation by oa dataset

try:
  deprivation_oa = pd.read_csv('lbth_census_2021_deprivation_oa.csv')
except:
  deprivation_oa = pd.read_csv('https://www.nomisweb.co.uk/api/v01/dataset/NM_2031_1.data.csv?date=latest&geography=629165479...629165982,629303674...629303812,629317322...629317326,629317336...629317343,629317349...629317360,629317362...629317366,629317371,629317374,629317376,629317378,629317379,629317381,629317385,629317388...629317392,629317394...629317397,629317399...629317403,629317405...629317407,629317409,629317411,629317412,629317416...629317420,629317422...629317424,629317426,629317429...629317434,629317436,629317437,629317440...629317442,629317444...629317450,629317452...629317456,629317459...629317461,629317463,629317466...629317468,629317472,629317474...629317479,629317481,629317483,629317486,629317487,629317490,629317492,629317494,629317495,629317497,629317499...629317502,629317504,629317505,629317507...629317509,629317512...629317514,629317517,629317520,629317523,629317525,629317527,629317531,629317534,629317535,629317537,629317538,629317540,629317543...629317548,629317551,629317554,629317555,629317558,629317560,629317562,629317563,629317565,629317566,629317569...629317573,629317576...629317578,629317580,629317582,629317584,629317585,629317587,629317590...629317592,629317594,629317596...629317599,629317601,629317603,629317604,629317606,629317607,629317610,629317612,629317613,629317615...629317617,629317619...629317621,629317623,629317625...629317629,629317631...629317634,629317636,629317640,629317648,629317650...629317659,629317662,629317663,629317666...629317669,629317672...629317687,629317689,629317691,629317694,629317695,629317697,629317698,629317700,629317701,629317703,629317704,629317706,629317707,629317711...629317714,629317716,629317718...629317720,629317722...629317724,629317726,629317729,629317731,629317733,629317736...629317742,629317744,629317746...629317748,629323624,629323625&c2021_dep_6=1...5&measures=20301')
  deprivation_oa.to_csv('lbth_census_2021_deprivation_oa.csv')

#merge deprivation data with spatial data
#deprivation_merge=merge_spatial_data(merged_wd_oa, deprivation_oa,"OA21CD", "GEOGRAPHY_CODE")

In [6]:
#read in relative deprivation 2021 dataset

try:
  deprivation_2021 = pd.read_csv('lbth_census_2021_deprivation_2021.csv')
except:
  deprivation_2021 = pd.read_csv('https://www.nomisweb.co.uk/api/v01/dataset/NM_2031_1.data.csv?date=latest&geography=645923010,2092957699,2013265927&c2021_dep_6=1...5&measures=20301')
  deprivation_2021.to_csv('lbth_census_2021_deprivation_2021.csv')

In [7]:
#read in relative deprivation 2021 dataset

try:
 deprivation_2011 = pd.read_csv('lbth_census_2021_deprivation_2011.csv')
except:
 deprivation_2011 = pd.read_csv('https://www.nomisweb.co.uk/api/v01/dataset/NM_519_1.data.csv?date=latest&geography=1946157257,2092957699,2013265927&rural_urban=0&c_deprived=1...5&measures=20100')
 deprivation_2011.to_csv('lbth_census_2021_deprivation_2011.csv')

In [8]:
#convert value to percent in 2011 dataset

deprivation_2011['OBS_VALUE'] = (
    np.select(
        condlist=[deprivation_2011['GEOGRAPHY_NAME'] == 'Tower Hamlets', deprivation_2011['GEOGRAPHY_NAME'] =='England', deprivation_2011['GEOGRAPHY_NAME'] =='London'],
        choicelist=[(deprivation_2011['OBS_VALUE']/101257)*100,(deprivation_2011['OBS_VALUE']/22063368)*100,(deprivation_2011['OBS_VALUE']/3266173)*100] 
        ))

In [9]:
#rename column deprivation column to allow for concatenate

deprivation_2011.rename(columns={'C_DEPRIVED_NAME': 'C2021_DEP_6_NAME'}, inplace=True)

In [10]:
#concatenate 2011 and 2021 dataframes 

deprivation_2011_2021=pd.concat([deprivation_2011,deprivation_2021])

In [11]:
#C2021_DEP_6_NAME column has different names for deprivation indexes-- numbers/ letters 

pd.options.display.max_columns = None

display(deprivation_2011_2021)

,Unnamed: 0,DATE,DATE_NAME,DATE_CODE,DATE_TYPE,DATE_TYPECODE,DATE_SORTORDER,GEOGRAPHY,GEOGRAPHY_NAME,GEOGRAPHY_CODE,GEOGRAPHY_TYPE,GEOGRAPHY_TYPECODE,GEOGRAPHY_SORTORDER,RURAL_URBAN,RURAL_URBAN_NAME,RURAL_URBAN_CODE,RURAL_URBAN_TYPE,RURAL_URBAN_TYPECODE,RURAL_URBAN_SORTORDER,C_DEPRIVED,C2021_DEP_6_NAME,C_DEPRIVED_CODE,C_DEPRIVED_TYPE,C_DEPRIVED_TYPECODE,C_DEPRIVED_SORTORDER,MEASURES,MEASURES_NAME,OBS_VALUE,OBS_STATUS,OBS_STATUS_NAME,OBS_CONF,OBS_CONF_NAME,URN,RECORD_OFFSET,RECORD_COUNT,C2021_DEP_6,C2021_DEP_6_CODE,C2021_DEP_6_TYPE,C2021_DEP_6_TYPECODE,C2021_DEP_6_SORTORDER
0,0,2011,2011,2011,date,0,0,1946157257,Tower Hamlets,E09000030,local authorities: district / unitary (prior t...,464,0,0.0,Total,0.0,2011 rural-urban classification,2000.0,0.0,1.0,Household is not deprived in any dimension,C_DEPRIVED_1,Household Deprivation,1000000.0,0.0,20100,Value,32.732552,A,Normal Value,F,Free (free for publication),Nm-519d1d32176e0d1946157257d0d1d20100,0,15,NaN,NaN,NaN,NaN,NaN
1,1,2011,2011,2011,date,0,0,1946157257,Tower Hamlets,E09000030,local authorities: district / unitary (prior t...,464,0,0.0,Total,0.0,2011 rural-urban classification,2000.0,0.0,2.0,Household is deprived in 1 dimension,C_DEPRIVED_2,Household Deprivation,1000000.0,1.0,20100,Value,35.749627,A,Normal Value,F,Free (free for publication),Nm-519d1d32176e0d1946157257d0d2d20100,1,15,NaN,NaN,NaN,NaN,NaN
2,2,2011,2011,2011,date,0,0,1946157257,Tower Hamlets,E09000030,local authorities: district / unitary (prior t...,464,0,0.0,Total,0.0,2011 rural-urban classification,2000.0,0.0,3.0,Household is deprived in 2 dimensions,C_DEPRIVED_3,Household Deprivation,1000000.0,2.0,20100,Value,20.726468,A,Normal Value,F,Free (free for publication),Nm-519d1d32176e0d1946157257d0d3d20100,2,15,NaN,NaN,NaN,NaN,NaN
3,3,2011,2011,2011,date,0,0,1946157257,Tower Hamlets,E09000030,local authorities: district / unitary (prior t...,464,0,0.0,Total,0.0,2011 rural-urban classification,2000.0,0.0,4.0,Household is deprived in 3 dimensions,C_DEPRIVED_4,Household Deprivation,1000000.0,3.0,20100,Value,9.314912,A,Normal Value,F,Free (free for publication),Nm-519d1d32176e0d1946157257d0d4d20100,3,15,NaN,NaN,NaN,NaN,NaN
4,4,2011,2011,2011,date,0,0,1946157257,Tower Hamlets,E09000030,local authorities: district / unitary (prior t...,464,0,0.0,Total,0.0,2011 rural-urban classification,2000.0,0.0,5.0,Household is deprived in 4 dimensions,C_DEPRIVED_5,Household Deprivation,1000000.0,4.0,20100,Value,1.476441,A,Normal Value,F,Free (free for publication),Nm-519d1d32176e0d1946157257d0d5d20100,4,15,NaN,NaN,NaN,NaN,NaN
5,5,2011,2011,2011,date,0,0,2092957699,England,E92000001,countries,499,1,0.0,Total,0.0,2011 rural-urban classification,2000.0,0.0,1.0,Household is not deprived in any dimension,C_DEPRIVED_1,Household Deprivation,1000000.0,0.0,20100,Value,42.539507,A,Normal Value,F,Free (free for publication),Nm-519d1d32176e0d2092957699d0d1d20100,5,15,NaN,NaN,NaN,NaN,NaN
6,6,2011,2011,2011,date,0,0,2092957699,England,E92000001,countries,499,1,0.0,Total,0.0,2011 rural-urban classification,2000.0,0.0,2.0,Household is deprived in 1 dimension,C_DEPRIVED_2,Household Deprivation,1000000.0,1.0,20100,Value,32.652227,A,Normal Value,F,Free (free for publication),Nm-519d1d32176e0d2092957699d0d2d20100,6,15,NaN,NaN,NaN,NaN,NaN
7,7,2011,2011,2011,date,0,0,2092957699,England,E92000001,countries,499,1,0.0,Total,0.0,2011 rural-urban classification,2000.0,0.0,3.0,Household is deprived in 2 dimensions,C_DEPRIVED_3,Household Deprivation,1000000.0,2.0,20100,Value,19.144774,A,Normal Value,F,Free (free for publication),Nm-519d1d32176e0d2092957699d0d3d20100,7,15,NaN,NaN,NaN,NaN,NaN
8,8,2011,2011,2011,date,0,0,2092957699,England,E92000001,countries,499,1,0.0,Total,0.0,2011 rural-urban classification,2000.0,0.0,4.0,Household is deprived in 3 dimensions,C_DEPRIVED_4,Household Deprivation,1000000.0,3.0,20100,Value,5.138028,A,Normal Value,F,Free (free for publication),Nm-519d1d32176e0d2092957699d0d4d20100,8,15,NaN,NaN,NaN,NaN,NaN
9,9,2011,2011,2011,date,0,

In [18]:
# Replace multiple substrings

deprivation_2011_2021= deprivation_2011_2021.apply(lambda x: x.replace({'one':'1','two': '2',
    'three': '3','four': '4'},regex=True))

display(deprivation_2011_2021)


,Unnamed: 0,DATE,DATE_NAME,DATE_CODE,DATE_TYPE,DATE_TYPECODE,DATE_SORTORDER,GEOGRAPHY,GEOGRAPHY_NAME,GEOGRAPHY_CODE,GEOGRAPHY_TYPE,GEOGRAPHY_TYPECODE,GEOGRAPHY_SORTORDER,RURAL_URBAN,RURAL_URBAN_NAME,RURAL_URBAN_CODE,RURAL_URBAN_TYPE,RURAL_URBAN_TYPECODE,RURAL_URBAN_SORTORDER,C_DEPRIVED,C2021_DEP_6_NAME,C_DEPRIVED_CODE,C_DEPRIVED_TYPE,C_DEPRIVED_TYPECODE,C_DEPRIVED_SORTORDER,MEASURES,MEASURES_NAME,OBS_VALUE,OBS_STATUS,OBS_STATUS_NAME,OBS_CONF,OBS_CONF_NAME,URN,RECORD_OFFSET,RECORD_COUNT,C2021_DEP_6,C2021_DEP_6_CODE,C2021_DEP_6_TYPE,C2021_DEP_6_TYPECODE,C2021_DEP_6_SORTORDER
0,0,2011,2011,2011,date,0,0,1946157257,Tower Hamlets,E09000030,local authorities: district / unitary (prior t...,464,0,0.0,Total,0.0,2011 rural-urban classification,2000.0,0.0,1.0,Household is not deprived in any dimension,C_DEPRIVED_1,Household Deprivation,1000000.0,0.0,20100,Value,32.732552,A,Normal Value,F,Free (free for publication),Nm-519d1d32176e0d1946157257d0d1d20100,0,15,NaN,NaN,NaN,NaN,NaN
1,1,2011,2011,2011,date,0,0,1946157257,Tower Hamlets,E09000030,local authorities: district / unitary (prior t...,464,0,0.0,Total,0.0,2011 rural-urban classification,2000.0,0.0,2.0,Household is deprived in 1 dimension,C_DEPRIVED_2,Household Deprivation,1000000.0,1.0,20100,Value,35.749627,A,Normal Value,F,Free (free for publication),Nm-519d1d32176e0d1946157257d0d2d20100,1,15,NaN,NaN,NaN,NaN,NaN
2,2,2011,2011,2011,date,0,0,1946157257,Tower Hamlets,E09000030,local authorities: district / unitary (prior t...,464,0,0.0,Total,0.0,2011 rural-urban classification,2000.0,0.0,3.0,Household is deprived in 2 dimensions,C_DEPRIVED_3,Household Deprivation,1000000.0,2.0,20100,Value,20.726468,A,Normal Value,F,Free (free for publication),Nm-519d1d32176e0d1946157257d0d3d20100,2,15,NaN,NaN,NaN,NaN,NaN
3,3,2011,2011,2011,date,0,0,1946157257,Tower Hamlets,E09000030,local authorities: district / unitary (prior t...,464,0,0.0,Total,0.0,2011 rural-urban classification,2000.0,0.0,4.0,Household is deprived in 3 dimensions,C_DEPRIVED_4,Household Deprivation,1000000.0,3.0,20100,Value,9.314912,A,Normal Value,F,Free (free for publication),Nm-519d1d32176e0d1946157257d0d4d20100,3,15,NaN,NaN,NaN,NaN,NaN
4,4,2011,2011,2011,date,0,0,1946157257,Tower Hamlets,E09000030,local authorities: district / unitary (prior t...,464,0,0.0,Total,0.0,2011 rural-urban classification,2000.0,0.0,5.0,Household is deprived in 4 dimensions,C_DEPRIVED_5,Household Deprivation,1000000.0,4.0,20100,Value,1.476441,A,Normal Value,F,Free (free for publication),Nm-519d1d32176e0d1946157257d0d5d20100,4,15,NaN,NaN,NaN,NaN,NaN
5,5,2011,2011,2011,date,0,0,2092957699,England,E92000001,countries,499,1,0.0,Total,0.0,2011 rural-urban classification,2000.0,0.0,1.0,Household is not deprived in any dimension,C_DEPRIVED_1,Household Deprivation,1000000.0,0.0,20100,Value,42.539507,A,Normal Value,F,Free (free for publication),Nm-519d1d32176e0d2092957699d0d1d20100,5,15,NaN,NaN,NaN,NaN,NaN
6,6,2011,2011,2011,date,0,0,2092957699,England,E92000001,countries,499,1,0.0,Total,0.0,2011 rural-urban classification,2000.0,0.0,2.0,Household is deprived in 1 dimension,C_DEPRIVED_2,Household Deprivation,1000000.0,1.0,20100,Value,32.652227,A,Normal Value,F,Free (free for publication),Nm-519d1d32176e0d2092957699d0d2d20100,6,15,NaN,NaN,NaN,NaN,NaN
7,7,2011,2011,2011,date,0,0,2092957699,England,E92000001,countries,499,1,0.0,Total,0.0,2011 rural-urban classification,2000.0,0.0,3.0,Household is deprived in 2 dimensions,C_DEPRIVED_3,Household Deprivation,1000000.0,2.0,20100,Value,19.144774,A,Normal Value,F,Free (free for publication),Nm-519d1d32176e0d2092957699d0d3d20100,7,15,NaN,NaN,NaN,NaN,NaN
8,8,2011,2011,2011,date,0,0,2092957699,England,E92000001,countries,499,1,0.0,Total,0.0,2011 rural-urban classification,2000.0,0.0,4.0,Household is deprived in 3 dimensions,C_DEPRIVED_4,Household Deprivation,1000000.0,3.0,20100,Value,5.138028,A,Normal Value,F,Free (free for publication),Nm-519d1d32176e0d2092957699d0d4d20100,8,15,NaN,NaN,NaN,NaN,NaN
9,9,2011,2011,2011,date,0,

In [24]:
#split dataframe by deprivation index

any_dimension= deprivation_2011_2021[deprivation_2011_2021['C2021_DEP_6_NAME'].str.contains('any dimension',na=False)].reset_index(drop=True)

one_dimension=deprivation_2011_2021[deprivation_2011_2021['C2021_DEP_6_NAME'].str.contains('1',na=False)].reset_index(drop=True)

two_dimension=deprivation_2011_2021[deprivation_2011_2021['C2021_DEP_6_NAME'].str.contains('2',na=False)].reset_index(drop=True)

three_dimension=deprivation_2011_2021[deprivation_2011_2021['C2021_DEP_6_NAME'].str.contains('3',na=False)].reset_index(drop=True)

four_dimension=deprivation_2011_2021[deprivation_2011_2021['C2021_DEP_6_NAME'].str.contains('4',na=False)].reset_index(drop=True)

display(four_dimension)

,Unnamed: 0,DATE,DATE_NAME,DATE_CODE,DATE_TYPE,DATE_TYPECODE,DATE_SORTORDER,GEOGRAPHY,GEOGRAPHY_NAME,GEOGRAPHY_CODE,GEOGRAPHY_TYPE,GEOGRAPHY_TYPECODE,GEOGRAPHY_SORTORDER,RURAL_URBAN,RURAL_URBAN_NAME,RURAL_URBAN_CODE,RURAL_URBAN_TYPE,RURAL_URBAN_TYPECODE,RURAL_URBAN_SORTORDER,C_DEPRIVED,C2021_DEP_6_NAME,C_DEPRIVED_CODE,C_DEPRIVED_TYPE,C_DEPRIVED_TYPECODE,C_DEPRIVED_SORTORDER,MEASURES,MEASURES_NAME,OBS_VALUE,OBS_STATUS,OBS_STATUS_NAME,OBS_CONF,OBS_CONF_NAME,URN,RECORD_OFFSET,RECORD_COUNT,C2021_DEP_6,C2021_DEP_6_CODE,C2021_DEP_6_TYPE,C2021_DEP_6_TYPECODE,C2021_DEP_6_SORTORDER
0,4,2011,2011,2011,date,0,0,1946157257,Tower Hamlets,E09000030,local authorities: district / unitary (prior t...,464,0,0.0,Total,0.0,2011 rural-urban classification,2000.0,0.0,5.0,Household is deprived in 4 dimensions,C_DEPRIVED_5,Household Deprivation,1000000.0,4.0,20100,Value,1.476441,A,Normal Value,F,Free (free for publication),Nm-519d1d32176e0d1946157257d0d5d20100,4,15,NaN,NaN,NaN,NaN,NaN
1,9,2011,2011,2011,date,0,0,2092957699,England,E92000001,countries,499,1,0.0,Total,0.0,2011 rural-urban classification,2000.0,0.0,5.0,Household is deprived in 4 dimensions,C_DEPRIVED_5,Household Deprivation,1000000.0,4.0,20100,Value,0.525464,A,Normal Value,F,Free (free for publication),Nm-519d1d32176e0d2092957699d0d5d20100,9,15,NaN,NaN,NaN,NaN,NaN
2,14,2011,2011,2011,date,0,0,2013265927,London,E12000007,regions,480,2,0.0,Total,0.0,2011 rural-urban classification,2000.0,0.0,5.0,Household is deprived in 4 dimensions,C_DEPRIVED_5,Household Deprivation,1000000.0,4.0,20100,Value,0.905249,A,Normal Value,F,Free (free for publication),Nm-519d1d32176e0d2013265927d0d5d20100,14,15,NaN,NaN,NaN,NaN,NaN
3,4,2021,2021,2021,date,0,0,645923010,Tower Hamlets,E09000030,2022 local authorities: districts,154,0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Household is deprived in 4 dimensions,NaN,NaN,NaN,NaN,20301,Percent,0.400000,A,Normal Value,F,Free (free for publication),Nm-2031d1d32336e0d645923010d5d20301,4,15,5.0,_5,Household deprivation,1000000.0,4.0
4,9,2021,2021,2021,date,0,0,2092957699,England,E92000001,countries,499,1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Household is deprived in 4 dimensions,NaN,NaN,NaN,NaN,20301,Percent,0.200000,A,Normal Value,F,Free (free for publication),Nm-2031d1d32336e0d2092957699d5d20301,9,15,5.0,_5,Household deprivation,1000000.0,4.0
5,14,2021,2021,2021,date,0,0,2013265927,London,E12000007,regions,480,2,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Household is deprived in 4 dimensions,NaN,NaN,NaN,NaN,20301,Percent,0.400000,A,Normal Value,F,Free (free for publication),Nm-2031d1d32336e0d2013265927d5d20301,14,15,5.0,_5,Household deprivation,1000000.0,4.0


In [69]:
# create a new plot with a title and axis labels
p = figure(title="Household is not deprived in any dimension", x_axis_label='Year', y_axis_label='Percentage')

source = ColumnDataSource(any_dimension)

for (name, group), color in zip(any_dimension.groupby('GEOGRAPHY_NAME'), Viridis3):
    p.line(x=group.DATE, y=group.OBS_VALUE, legend_label=str(name), color=color,line_width=3)
    
p.xaxis.ticker=[2011,2021] #customise x axis tick values 

p.xaxis.axis_label_text_font_size = "20pt"
p.xaxis.major_label_text_font_size = "15pt"
p.xaxis.axis_label_text_font = "arial"
p.xaxis.axis_label_text_color = "black"

p.yaxis.axis_label_text_font_size = "20pt"
p.yaxis.major_label_text_font_size = "15pt"
p.yaxis.axis_label_text_font = "arial"
p.yaxis.axis_label_text_color = "black"

p.legend.location = "bottom_right"
p.legend.label_text_font_size = "15pt"
p.legend.label_text_font = "arial"
p.legend.label_text_color = "black"

p.title.text_font_size = '15pt'

show(p)

In [14]:
with st.sidebar:
    add_radio = st.radio(label='Variable selection', options=('Household deprivation','Homelessness'))

if add_radio == "Household deprivation":
     st.title('Household deprivation')
     deprivation_radio= st.radio("Deprivation index", ('Household is not deprived in any dimension',
     'Household is deprived in one dimension',
     'Household is deprived in two dimensions',
     'Household is deprived in three dimensions',
     'Household is deprived in four dimensions'))

     if deprivation_radio =='Household is not deprived in any dimension': 
       plot_wards(deprivation_merge,column='C2021_DEP_6_NAME', string='Household is not deprived in any dimension',agg='mean',
       title='Percentage of Households')
       st.bokeh_chart(p, use_container_width=True)

     elif deprivation_radio =='Household is deprived in one dimension':  
       plot_wards(deprivation_merge,column='C2021_DEP_6_NAME', string='Household is deprived in one dimension',agg='mean',
       title='Percentage of Households')

     elif deprivation_radio =='Household is deprived in two dimensions': 
       plot_wards(deprivation_merge,column='C2021_DEP_6_NAME', string='Household is deprived in two dimensions',agg='mean',
       title='Percentage of Households')

     elif deprivation_radio =='Household is deprived in three dimensions': 
       plot_wards(deprivation_merge,column='C2021_DEP_6_NAME', string='Household is deprived in three dimensions',agg='mean',
       title='Percentage of Households')

     elif deprivation_radio== 'Household is deprived in four dimensions':
       plot_wards(deprivation_merge,column='C2021_DEP_6_NAME', string='Household is deprived in four dimensions',agg='mean',
       title='Percentage of Households')

2023-01-16 09:51:39.202 
  command:

    streamlit run C:\Users\sophie.porton\Anaconda3\lib\site-packages\ipykernel_launcher.py [ARGUMENTS]


NameError: name 'deprivation_merge' is not defined